In [1]:
#!pip install tensorboard
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as data
import torchvision.utils as utils
import os

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [2]:
#dataset path
data_path_train = "data/training"
data_path_test = "data/testing"

In [3]:
# data transform, you can add different transform methods and resize image to any size
img_size = 224
transform = transforms.Compose([
                       transforms.Resize((img_size,img_size)),
                       transforms.ToTensor()
                       ])

#build dataset
dataset = datasets.ImageFolder(root=data_path_train,transform=transform)

# spilt your data into train and val
TOTAL_SIZE = len(dataset)
ratio = 0.9
train_len = round(TOTAL_SIZE * ratio)
valid_len = round(TOTAL_SIZE * (1-ratio))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, valid_len])

#build dataloader
train_data_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True,  num_workers=4)
val_data_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=True,  num_workers=4)

#check dataset
print(dataset)
print(dataset.class_to_idx)

Dataset ImageFolder
    Number of datapoints: 1646
    Root location: data/training
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )
{'Baked Potato': 0, 'Crispy Chicken': 1, 'Donut': 2, 'Fries': 3}


/home/NFS/course/dl2023f/dl2023f_087/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
#train function
def train(model, criterion, optimizer):
    model.train()
    total_loss = 0.0
    total_correct = 0
    
    # Iterate over data
    for inputs, labels in train_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backward + optimize
        loss.backward()
        optimizer.step()

        # statistics
        total_loss += loss.item()
        total_correct += torch.sum(preds == labels.data)
        
    avg_loss = total_loss / len(train_data_loader)
    accuracy = total_correct.double() / len(train_dataset) * 100

    print('Training Accuracy: {:.4f}% Training Loss: {:.4f}'.format(accuracy, avg_loss))
    return 

#validation function
def valid(model, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    
    # Iterate over data
    for inputs, labels in val_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # statistics
        total_loss += loss.item()
        total_correct += torch.sum(preds == labels.data)
        
    avg_loss = total_loss / len(val_data_loader)
    accuracy = total_correct.double() / len(val_dataset) * 100

    print('Validation Accuracy: {:.4f}% Validation Loss: {:.4f}'.format(accuracy, avg_loss))
    return accuracy

In [5]:
# using gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#build your model here
model = models.resnet18(pretrained=True)

/home/NFS/course/dl2023f/dl2023f_087/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/NFS/course/dl2023f/dl2023f_087/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/NFS/course/dl2023f/dl2023f_087/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' a

In [6]:
####################  implement your optimizer ###################################
## you can use any training methods if you want (ex:lr decay, weight decay.....)
learning_rate = 0.01
epochs = 50
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9) 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# start training
model.to(device=device)
acc_best = 0.0

print('--------------start training--------------')
for epoch in range(1, epochs+1):
    
    print('epoch:', epoch)
    train(model, criterion, optimizer)
    accuracy = valid(model, criterion)
    writer.add_scalar('Learning_rate', learning_rate, epoch)
    writer.add_histogram('Weights', model.conv1.weight, epoch)
    writer.add_histogram('Gradients', model.conv1.weight.grad, epoch)
    if accuracy > acc_best:
        acc_best = accuracy
        print("model saved")
        torch.save(model, "model.pth")

writer.close()

--------------start training--------------
epoch: 1
Training Accuracy: 27.4814% Training Loss: 2.4538
Validation Accuracy: 29.6970% Validation Loss: 5.8419
model saved
epoch: 2
Training Accuracy: 36.5969% Training Loss: 1.5520
Validation Accuracy: 37.5758% Validation Loss: 2.7976
model saved
epoch: 3
Training Accuracy: 38.3525% Training Loss: 1.3528
Validation Accuracy: 37.5758% Validation Loss: 3.4826
epoch: 4
Training Accuracy: 42.8089% Training Loss: 1.3337
Validation Accuracy: 47.2727% Validation Loss: 1.3077
model saved
epoch: 5
Training Accuracy: 51.3167% Training Loss: 1.1407
Validation Accuracy: 44.2424% Validation Loss: 1.1752
epoch: 6
Training Accuracy: 56.1783% Training Loss: 1.0614
Validation Accuracy: 42.4242% Validation Loss: 2.9637
epoch: 7
Training Accuracy: 61.0398% Training Loss: 0.9945
Validation Accuracy: 60.6061% Validation Loss: 0.9770
model saved
epoch: 8
Training Accuracy: 59.5544% Training Loss: 1.0149
Validation Accuracy: 59.3939% Validation Loss: 0.9125
epoch

In [ ]:
transform_test = transforms.Compose([transforms.Resize((img_size,img_size)),
                                    transforms.ToTensor()
                                    ])

dataset_test = datasets.ImageFolder(root=data_path_test, transform=transform_test)
dataloader_test  = data.DataLoader(dataset_test, batch_size=8, shuffle=False, num_workers=4) 

In [ ]:
# load the model so that you don't need to train the model again
test_model = torch.load("model.pth").to(device)

In [ ]:
def test(model):
    with torch.no_grad():
        model.eval()
        bs = dataloader_test.batch_size
        result = []
        for i, (data, target) in enumerate(dataloader_test):
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, preds = torch.max(output, 1, keepdim=True)
            
            arr = preds.data.cpu().numpy()
            for j in range(preds.size()[0]):
                file_name = dataset_test.samples[i*bs+j][0].split('/')[-1]
                result.append((file_name,preds[j].cpu().numpy()[0]))
    return result

In [ ]:
result = test(test_model)

In [ ]:
with open ('ID_result.csv','w') as f:
    f.write('ID,label\n')
    for data in result:
        f.write(data[0]+','+str(data[1])+'\n')